# Start Gui

This notebook can be used to start the GUI and test out stuff.

In [13]:
# -*- coding: utf-8 -*-
"""
@author: Fabian Balzer

***
LICENSE:
    Copyright 2021 Fabian Balzer

   Licensed under the Apache License, Version 2.0 (the "License");
   you may not use this file except in compliance with the License.
   You may obtain a copy of the License at

     http://www.apache.org/licenses/LICENSE-2.0

   Unless required by applicable law or agreed to in writing, software
   distributed under the License is distributed on an "AS IS" BASIS,
   WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
   See the License for the specific language governing permissions and
   limitations under the License.
***

Code to open a GUI Dominion randomizer
"""
# %%

import sys
import random_kingdominion as rm
import PyQt5.QtWidgets as QW


def start_program():
    """A function to include everything needed to start the application"""
    # Check whether there is already a running QApplication (e.g. if running
    # from an IDE). This setup prevents crashes for the next run:
    qapp = QW.QApplication.instance()
    if not qapp:
        qapp = QW.QApplication(sys.argv)
    app = rm.UIMainWindow()  # creating the instance
    app.show()
    app.activateWindow()
    qapp.exec_()  # Start the Qt event loop



In [23]:

start_program()

In [6]:
import random_kingdominion as rk
import os
import pandas as pd
import numpy as np
from typing import Sequence


def read_dataframe_from_file(fpath: str, eval_lists=False):
    """Read a dataframe"""
    if os.path.isfile(fpath):
        df = pd.read_csv(fpath, sep=";", header=0)
        if eval_lists:
            for colname in df.columns:
                if "type" in colname.lower():
                    # Make sure we properly handle lists
                    df[colname] = df[colname].apply(eval)
    else:
        raise FileNotFoundError(
            2, "Couldn't find the raw card data file, please download it first."
        )
    return df
from functools import reduce
df = read_dataframe_from_file(rk.FPATH_CARD_DATA, True)
names = rk.ALL_CSOS["Name"].sample(200).values

535 µs ± 3.39 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
5.47 ms ± 99.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [1]:
from PyQt5.QtCore import Qt, QStringListModel, QPoint
from PyQt5.QtGui import QIcon, QStandardItemModel, QPalette, QTextCursor, QTextCharFormat, QCursor
from PyQt5.QtWidgets import QApplication, QWidget, QHBoxLayout, QLineEdit, QPushButton, QVBoxLayout, QMainWindow, QCompleter, QTextEdit,QLabel, QFrame
from typing import Sequence
from random_kingdominion.constants import ALL_CSOS




class MyMainWindow(QMainWindow):
    def __init__(self):
        super().__init__()
        self.initUI()

    def initUI(self):
        # Create a central widget
        centralWidget = QWidget(self)
        self.setCentralWidget(centralWidget)

        # Create a layout for the central widget
        layout = QVBoxLayout()

        # Create the ValidatingLineEditWidget
        self.validatingLineEditWidget = CSVValidationWidget()
        layout.addWidget(self.validatingLineEditWidget)
        self.validatingLineEditWidget.set_allowed_csos(set(ALL_CSOS.Name))

        # Set the layout for the central widget
        centralWidget.setLayout(layout)

        # Set window properties
        self.setWindowTitle('Main Window with ValidatingLineEditWidget')
        self.setGeometry(100, 100, 400, 200)

qapp = QApplication.instance()
if not qapp:
    qapp = QApplication(sys.argv)
app = MyMainWindow()  # creating the instance
app.show()
app.activateWindow()
qapp.exec_()  # Start the Qt event loop

0